In [14]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PyPDF2 import PdfReader
from sklearn.pipeline import Pipeline
from joblib import load
import re
import os
import logging
import threading
import shutil

logging.basicConfig(level=logging.DEBUG)

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    try:
        with open(pdf_file, 'rb') as file:
            reader = PdfReader(file)
            text = ''
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
        return text
    except Exception as e:
        logging.error(f"Error extracting text from PDF: {e}")
        return ""

# Function to clean the text
def clean_text(text):
    try:
        cleaned_text = re.sub(r'Page\s+\d+\s+of\s+\d+|Confidential|Header|Footer', '', text, flags=re.IGNORECASE)
        cleaned_text = cleaned_text.lower()
        cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
        return cleaned_text
    except Exception as e:
        logging.error(f"Error cleaning text: {e}")
        return text

# Function to preprocess resume, qualification, and experience text
def preprocess_text(resume):
    try:
        qualification_match = re.search(r'Qualification[s]?:\s*(.*?)\s*(Experience[s]?:|$)', resume, re.IGNORECASE)
        experience_match = re.search(r'Experience[s]?:\s*(.*?)\s*(Education[s]?:|$)', resume, re.IGNORECASE)

        qualification_text = qualification_match.group(1) if qualification_match else ''
        experience_text = experience_match.group(1) if experience_match else ''

        cleaned_resume = clean_text(resume)
        cleaned_qualification = clean_text(qualification_text)
        cleaned_experience = clean_text(experience_text)
        
        concatenated_text = cleaned_resume + ' ' + cleaned_qualification + ' ' + cleaned_experience
        return concatenated_text
    except Exception as e:
        logging.error(f"Error preprocessing text: {e}")
        return resume

# Load the pre-trained pipeline model
pipeline_file = 'pipeline.joblib'
loaded_pipeline = None
try:
    loaded_pipeline = load(pipeline_file)
    logging.info("Pipeline loaded successfully")
except Exception as e:
    logging.error(f"Error loading pipeline: {e}")

# Function to categorize the uploaded PDF
def categorize_pdf(file_path):
    resume_text = extract_text_from_pdf(file_path)
    processed_text = preprocess_text(resume_text)
    if loaded_pipeline:
        try:
            category = loaded_pipeline.predict([processed_text])[0]
            return category
        except Exception as e:
            logging.error(f"Error predicting category: {e}")
            return "Error predicting category"
    else:
        logging.error("Pipeline is not loaded")
        return "Pipeline not loaded"

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

ALLOWED_EXTENSIONS = {'pdf'}

class PDFUploaderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Resume Screener")
        self.root.geometry("600x400")
        
        # Adding a dark theme
        self.style = ttk.Style()
        self.style.theme_use("clam")  # Use 'clam' theme
        self.style.configure("TButton", padding=6, relief="flat", background="#007bff", foreground="#ffffff", bordercolor="#007bff")
        self.style.map("TButton", background=[("active", "#0069d9")])
        self.style.configure("TLabel", padding=6, background="#212121", foreground="#ffffff")  # Dark background, white text
        self.style.configure("TFrame", background="#212121")  # Dark background for frames
        self.style.configure("TText", background="#333333", foreground="#ffffff", font=("Helvetica", 12))  # Dark background, white text, larger font

        # Creating the main frame
        self.main_frame = ttk.Frame(root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

        # Frame for folder selection
        self.folder_frame = ttk.Frame(self.main_frame)
        self.folder_frame.pack(pady=10)

        # Select folder button
        self.select_folder_button = ttk.Button(self.folder_frame, text="Select Folder", command=self.select_folder)
        self.select_folder_button.pack(side=tk.LEFT, padx=5)

        # Upload folder button
        self.upload_button = ttk.Button(self.folder_frame, text="Upload Folder", command=self.upload_folder)
        self.upload_button.pack(side=tk.LEFT, padx=5)

        # Label to display the selected folder
        self.result_label = ttk.Label(self.main_frame, text="Select a folder to categorize PDFs", anchor=tk.CENTER)
        self.result_label.pack(pady=10, fill=tk.BOTH)

        # Text widget to display results
        self.results_text = tk.Text(self.main_frame, wrap=tk.WORD, state=tk.DISABLED, bg="#333333", fg="#ffffff", font=("Helvetica", 14))  # Dark background, white text, larger font
        self.results_text.pack(pady=10, fill=tk.BOTH, expand=True)

        self.selected_folder = None

       

    def select_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.selected_folder = folder
            self.result_label.config(text=f"Selected folder: {folder}")
        else:
            self.selected_folder = None
            self.result_label.config(text="No folder selected")

    def upload_folder(self):
        if not self.selected_folder:
            messagebox.showerror("Error", "No folder selected")
            return
        
        if os.path.isdir(self.selected_folder):
            self.upload_button.config(state=tk.DISABLED)  # Disable the button while processing
            threading.Thread(target=self.process_pdfs).start()
        else:
            messagebox.showerror("Error", "Selected path is not a folder")

    def process_pdfs(self):
        pdf_files = [os.path.join(self.selected_folder, f) for f in os.listdir(self.selected_folder) if allowed_file(f)]
        if not pdf_files:
            messagebox.showerror("Error", "No PDF files found in the selected folder")
            self.upload_button.config(state=tk.NORMAL)  # Re-enable the button after processing
            return
        
        results = []
        parent_folder = 'C:/Users/Kainat Ahmed/Desktop/Resume-Screener/Categorized_PDFs'
        os.makedirs(parent_folder, exist_ok=True)

        for pdf_file in pdf_files:
            category = categorize_pdf(pdf_file)
            category_folder = os.path.join(parent_folder, category)
            os.makedirs(category_folder, exist_ok=True)
            shutil.copy(pdf_file, category_folder)
            results.append(f'{os.path.basename(pdf_file)}: {category}')
        
        results_text = '\n'.join(results)
        self.result_label.config(text="Uploaded and Categorized PDFs:")
        self.update_results_text(results_text)
        self.display_folders(parent_folder)
        self.upload_button.config(state=tk.NORMAL)  # Re-enable the button after processing

    def update_results_text(self, text):
        self.results_text.config(state=tk.NORMAL)
        self.results_text.delete(1.0, tk.END)
        self.results_text.insert(tk.END, text)
        self.results_text.config(state=tk.DISABLED)

    def display_folders(self, parent_folder):
        folder_details = []
        for folder_name in os.listdir(parent_folder):
            folder_path = os.path.join(parent_folder, folder_name)
            if os.path.isdir(folder_path):
                num_files = len([f for f in os.listdir(folder_path) if allowed_file(f)])
                folder_details.append(f'{folder_name}: {num_files} resumes')

        folder_details_text = '\n'.join(folder_details)
        self.update_results_text(folder_details_text)

if __name__ == '__main__':
    root = tk.Tk()
    app = PDFUploaderApp(root)
    root.mainloop()


INFO:root:Pipeline loaded successfully
ERROR:root:Error predicting category: Expected 2D array, got 1D array instead:
array=['\n    name michael johnson\n    job title software engineer\n    company creative minds\n\n    education\n    bfa in graphic design columbia university\n\n    skills\n    financial modeling adobe photoshop data analysis employee relations salesforce\n\n    experience\n     worked as a software engineer at creative minds for 3 years\n     led projects and collaborated with team members to achieve company goals\n     utilized skills such as financial modeling adobe photoshop data analysis to improve processes and increase efficiency\n      '].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
ERROR:root:Error predicting category: Expected 2D array, got 1D array instead:
array=['\n    name laura hernandez\n    job title financial analyst\n    company innovatex\n\n    educati